In [2]:
import os 

In [3]:
def list_files(directory):
    file_list = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
            
    return file_list

In [4]:
user_pdf_list = list_files('pewCombinedpdf')

len(user_pdf_list)

548

In [15]:
type(user_pdf_list)

list

In [6]:
from langchain.schema import Document

def create_docs(user_pdf_list):
    docs = []
    for filename in user_pdf_list:
        if not filename.lower().endswith(".pdf"):
            continue
        else:
            pdf_data = get_pdf_text(filename)
            docs.append(Document(
                page_content=pdf_data
            ))
    return docs

In [7]:
os.environ["PINECONE_API_KEY"] = "PINECONE_API_KEY"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [8]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_community.vectorstores import Pinecone as PineconeStore
from langchain.vectorstores import Pinecone
from pinecone import Pinecone

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [13]:
data = create_docs(user_pdf_list)

In [12]:
from pypdf import PdfReader

In [11]:
def get_pdf_text(pdf_doc):
    text = ""
    pdf_reader = PdfReader(pdf_doc)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [16]:
data[:4]

[Document(page_content='Space System Architecture \nLecture 1: Space Systems and Definitions \nFraming Document by Annalisa Weigel \nRequired reading: \na) Framing document (01010 week_1 framing.pdf)\nb) ESD Architecture document (01020_ESD_Sys_Arch.pdf)\nc) Rectin & Maier – Chaps 1 & 2 (01030Rechtin_and_Maier_Ch_1_2.pdf)\nd) Dave Clark on Internet Architecture (01031 NatureOfArch-1.pdf)\ne) Maier on System of Systems: Mark W. Maier, “Architecting principles for\nsystems-of-systems,” Systems Engineering , Vol. 1, No. 4, pp 267-284, 1998. \n(http://www3.interscience.wiley.com/cgi-bin/abstract/40003635/ABSTRACT) \nf)\tSage & Lynch on System Views: Andrew. P. Sage and Charles L. Lynch, “Systems integration and architecting: An overview of principles, practices, and \nperspectives,” Systems Engineering , Vol. 1, No. 3, pp 176-227, 1998. \n(http://www3.interscience.wiley.com/cgi-bin/abstract/30002173/ABSTRACT). \ng) Iridium Trades: Garrison, T. P., M. Ince, J. Pizzicaroli, and P. A. Swan, “

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def push_to_pinecone(pinecone_apikey,pinecone_environment,pinecone_index_name,embeddings,docs):
    text_splitter = RecursiveCharacterTextSplitter()
    document_chunks = text_splitter.split_documents(docs)
    pinecone = Pinecone(
        api_key=pinecone_apikey,environment=pinecone_environment
        )
    # create a vectorstore from the chunks
    vector_store=PineconeStore.from_documents(document_chunks,embeddings,index_name=pinecone_index_name)

In [ ]:
push_to_pinecone(os.environ['PINECONE_API_KEY'], "gcp-starter", "ccsv", embeddings, data)